# Traffic Data - TOM TOM

In [11]:
import requests
import folium
import numpy as np

# Set up your API key and endpoint
API_KEY = 'GmBsjR7eOfcU0JM2vmjwH8OsSXPmPLAs'

# TomTom Traffic Flow API URL
url = f"https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"

# Exact coordinates of the point in Ongar (converted to decimal degrees)
latitude = 53.393167
longitude = -6.441028

# Function to generate a grid of points around a central point (lat, lon) within a radius
def generate_grid(center_lat, center_lon, radius_km, num_points):
    # Earth radius in kilometers
    R = 6371.0
    # Convert radius to degrees
    delta_lat = radius_km / R * (180 / np.pi)
    delta_lon = radius_km / (R * np.cos(center_lat * np.pi / 180)) * (180 / np.pi)

    # Create a grid of points
    latitudes = np.linspace(center_lat - delta_lat, center_lat + delta_lat, num_points)
    longitudes = np.linspace(center_lon - delta_lon, center_lon + delta_lon, num_points)
    
    grid_points = [(lat, lon) for lat in latitudes for lon in longitudes]
    return grid_points

# Function to fetch traffic flow data for a given point (lat, lon)
def get_traffic_data(lat, lon):
    params = {
        'point': f'{lat},{lon}',  # Coordinates of the point
        'unit': 'KMPH',           # Use km/h for speed
        'key': API_KEY
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data for point ({lat}, {lon}): {response.status_code}")
        return None

# Function to get a color based on traffic congestion
def get_traffic_color(current_speed, free_flow_speed):
    # Avoid division by zero
    if free_flow_speed == 0:
        return 'black'  # Error case
    
    # Ratio of current speed to free flow speed
    speed_ratio = current_speed / free_flow_speed
    
    # Assign colors based on speed ratio
    if speed_ratio > 0.8:
        return 'green'   # Little to no congestion
    elif speed_ratio > 0.5:
        return 'yellow'  # Moderate congestion
    else:
        return 'red'     # Heavy congestion

# Generate a grid of points around Ongar with a 1 km radius and 10 points in each direction
grid_points = generate_grid(latitude, longitude, radius_km=0.1, num_points=10)
print(f"Generated {len(grid_points)} grid points around ({latitude}, {longitude})")

# Create a map centered around Ongar
ongar_map = folium.Map(location=[latitude, longitude], zoom_start=14)

# Loop through each point in the grid and fetch traffic data
for lat, lon in grid_points:
    print(f"Fetching traffic data for point ({lat}, {lon})...")
    data = get_traffic_data(lat, lon)
    
    if data:
        flow_data = data.get('flowSegmentData', {})
        current_speed = flow_data.get('currentSpeed', 0)
        free_flow_speed = flow_data.get('freeFlowSpeed', 1)  # Avoid division by zero

        # Debug information
        print(f"Current Speed: {current_speed} km/h, Free Flow Speed: {free_flow_speed} km/h")

        # Get color based on traffic conditions
        line_color = get_traffic_color(current_speed, free_flow_speed)
        print(f"Line Color: {line_color}")
        
        coordinates = flow_data.get('coordinates', {}).get('coordinate', [])
        # If there are coordinates, add them to the map
        if coordinates:
            route_coords = [(coord['latitude'], coord['longitude']) for coord in coordinates]
            # Add the traffic route to the map with dynamic color based on traffic
            folium.PolyLine(route_coords, color=line_color, weight=5, opacity=0.7).add_to(ongar_map)
            print("Traffic route added to the map.")
        else:
            print("No coordinates found for this point.")
    else:
        print(f"No data returned for point ({lat}, {lon}).")

# Save the map to an HTML file
output_map_path = "ongar_traffic_dynamic_map.html"
ongar_map.save(output_map_path)

print(f"Map showing dynamic traffic flow for all roads in Ongar saved to {output_map_path}")


Generated 100 grid points around (53.393167, -6.441028)
Fetching traffic data for point (53.39226767839408, -6.442536117589252)...
Current Speed: 49 km/h, Free Flow Speed: 49 km/h
Line Color: green
Traffic route added to the map.
Fetching traffic data for point (53.39226767839408, -6.442200980347196)...
Current Speed: 49 km/h, Free Flow Speed: 49 km/h
Line Color: green
Traffic route added to the map.
Fetching traffic data for point (53.39226767839408, -6.44186584310514)...
Current Speed: 49 km/h, Free Flow Speed: 49 km/h
Line Color: green
Traffic route added to the map.
Fetching traffic data for point (53.39226767839408, -6.441530705863084)...
Current Speed: 49 km/h, Free Flow Speed: 49 km/h
Line Color: green
Traffic route added to the map.
Fetching traffic data for point (53.39226767839408, -6.441195568621028)...
Current Speed: 49 km/h, Free Flow Speed: 49 km/h
Line Color: green
Traffic route added to the map.
Fetching traffic data for point (53.39226767839408, -6.440860431378972)...


# AIR Quality API 

In [12]:
import requests

# Set up your API token and endpoint
API_TOKEN = 'c9fe66adf13aab2b8f0ccae86f8f2d4fe0f9724b'
AQI_URL = f'https://api.waqi.info/feed/@13365/?token={API_TOKEN}'

def fetch_air_quality(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        data = response.json()
        
        if data['status'] == 'ok':
            return data['data']
        else:
            print("Error:", data.get('data', 'Unknown error'))
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None

def display_air_quality(data):
    if data:
        aqi = data['aqi']
        dominant_pollutant = data['dominentpol']
        time = data['time']['s']
        city = data['city']['name']

        print(f"Air Quality Index (AQI) for {city}: {aqi}")
        print(f"Dominant Pollutant: {dominant_pollutant}")
        print(f"Data as of: {time}")

        # Display individual pollutant levels
        iaqi = data['iaqi']
        print("Pollutant Levels:")
        for pollutant, values in iaqi.items():
            print(f"  {pollutant}: {values['v']}")

        # Forecast
        if 'forecast' in data:
            print("\nForecast:")
            for key, daily_data in data['forecast']['daily'].items():
                print(f"  {key}:")
                for day in daily_data:
                    print(f"    {day['day']}: Avg: {day['avg']}, Max: {day['max']}, Min: {day['min']}")
    else:
        print("No air quality data available.")

# Fetch and display the air quality data
air_quality_data = fetch_air_quality(AQI_URL)
display_air_quality(air_quality_data)


Air Quality Index (AQI) for Blanchardstown, Ireland: 41
Dominant Pollutant: pm25
Data as of: 2024-09-20 09:00:00
Pollutant Levels:
  dew: 13
  h: 93
  no2: 22.5
  p: 1026
  pm10: 20
  pm25: 41
  t: 14
  w: 6.1
  wg: 8.2

Forecast:
  o3:
    2024-09-18: Avg: 6, Max: 16, Min: 1
    2024-09-19: Avg: 5, Max: 15, Min: 1
    2024-09-20: Avg: 9, Max: 16, Min: 3
    2024-09-21: Avg: 16, Max: 18, Min: 13
    2024-09-22: Avg: 15, Max: 17, Min: 14
    2024-09-23: Avg: 9, Max: 16, Min: 5
    2024-09-24: Avg: 8, Max: 14, Min: 4
    2024-09-25: Avg: 12, Max: 13, Min: 12
  pm10:
    2024-09-18: Avg: 18, Max: 21, Min: 11
    2024-09-19: Avg: 21, Max: 25, Min: 17
    2024-09-20: Avg: 15, Max: 17, Min: 13
    2024-09-21: Avg: 21, Max: 27, Min: 13
    2024-09-22: Avg: 24, Max: 29, Min: 18
    2024-09-23: Avg: 15, Max: 21, Min: 11
    2024-09-24: Avg: 12, Max: 14, Min: 9
    2024-09-25: Avg: 11, Max: 11, Min: 10
  pm25:
    2024-09-18: Avg: 32, Max: 42, Min: 17
    2024-09-19: Avg: 35, Max: 47, Min: 22
  

# GFTS Public Transport

In [ ]:
import requests
from google.transit import gtfs_realtime_pb2  # Protobuf bindings for GTFS-RT

# API endpoint and your API key
api_url = "https://api.nationaltransport.ie/gtfsr/v2/gtfsr"
api_key = "47ffa50805124a9194d24a21fdae89f9"

# Add the headers with the API key
headers = {
    'x-api-key': api_key
}

# List of stop IDs we are interested in
target_stop_ids = ["7700", "7699", "7048", "7047", "7159", "7160"]

# Make the API call
response = requests.get(api_url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Decode the GTFS-RT protobuf response
    feed = gtfs_realtime_pb2.FeedMessage()
    try:
        feed.ParseFromString(response.content)
        print("Successfully parsed GTFS-RT feed!")
        
        trip_update_count = 0
        vehicle_position_count = 0
        found_vehicles = False  # To track if we find any vehicles for the target stops

        # Loop through the feed to find vehicles related to the target stop IDs
        for entity in feed.entity:
            if entity.HasField('trip_update'):
                trip_update_count += 1
                trip_update = entity.trip_update
                # Loop through all stop time updates
                for stop_time_update in trip_update.stop_time_update:
                    # Print all stop_ids encountered for debugging
                    print(f"Encountered Stop ID: {stop_time_update.stop_id}")

                    if stop_time_update.stop_id in target_stop_ids:
                        found_vehicles = True
                        print(f"Vehicle related to Stop ID {stop_time_update.stop_id}:")
                        print("Trip ID:", trip_update.trip.trip_id if trip_update.trip.HasField('trip_id') else "Unknown")
                        print("Arrival Time:", stop_time_update.arrival.time if stop_time_update.HasField('arrival') else "Unknown")
                        print("Departure Time:", stop_time_update.departure.time if stop_time_update.HasField('departure') else "Unknown")
                        print("Stop Sequence:", stop_time_update.stop_sequence)
                        print("------")
            elif entity.HasField('vehicle'):
                vehicle_position_count += 1
                vehicle = entity.vehicle
                # Check if the vehicle is at or near one of the target stops
                if vehicle.HasField('stop_id'):
                    # Print all stop_ids encountered for debugging
                    print(f"Encountered Vehicle Stop ID: {vehicle.stop_id}")
                    if vehicle.stop_id in target_stop_ids:
                        found_vehicles = True
                        print(f"Vehicle at Stop ID {vehicle.stop_id}:")
                        print("Vehicle ID:", vehicle.vehicle.id if vehicle.HasField('vehicle') else "Unknown")
                        print("Trip ID:", vehicle.trip.trip_id if vehicle.HasField('trip') else "Unknown")
                        print("Current Status:", vehicle.current_status)
                        print("Position:", f"Lat: {vehicle.position.latitude}, Lon: {vehicle.position.longitude}")
                        print("Congestion Level:", vehicle.congestion_level if vehicle.HasField('congestion_level') else "Not available")
                        print("------")
        
        print(f"Parsed {trip_update_count} trip updates and {vehicle_position_count} vehicle positions.")
        if not found_vehicles:
            print(f"No vehicles found for the target stop IDs: {', '.join(target_stop_ids)}.")
    
    except Exception as e:
        print(f"Failed to parse GTFS-RT feed: {e}")
else:
    print(f"Failed to retrieve data. HTTP Status code: {response.status_code}")
    print("Response content:", response.content)
